In [1]:
import requests

In [2]:
url = ('https://fbref.com/en/comps/12/La-Liga-Stats')

In [3]:
data = requests.get(url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [6]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [7]:
data = requests.get(team_urls[0])

In [8]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

/home/emmanuel/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [9]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [10]:
data = requests.get(f"https://fbref.com{links[0]}")

In [11]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [12]:
shooting.head()

For Barcelona                                                            \
           Date   Time          Comp        Round  Day Venue Result GF GA   
0    2022-08-13  21:00       La Liga  Matchweek 1  Sat  Home      D  0  0   
1    2022-08-21  22:00       La Liga  Matchweek 2  Sun  Away      W  4  1   
2    2022-08-28  19:30       La Liga  Matchweek 3  Sun  Home      W  4  0   
3    2022-09-03  21:00       La Liga  Matchweek 4  Sat  Away      W  3  0   
4    2022-09-07  21:00  Champions Lg  Group stage  Wed  Home      W  5  1   

                      ... Standard               Expected                    \
            Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh G-xG   
0     Rayo Vallecano  ...     17.0  1.0  0     0      1.9  1.9    0.09 -1.9   
1      Real Sociedad  ...     14.6  0.0  0     0      2.1  2.1    0.14  1.9   
2         Valladolid  ...     14.4  1.0  0     0      2.6  2.6    0.12  1.4   
3            Sevilla  ...     16.0  2.0  0     0      4.0  4.0    0.24 -1.0   
4  cz Viktoria Plzeň  ...     16.2  0.0  0     0      2.3  2.3    0.11  2.7   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0    -1.9        Match Report  
1     1.9        Match Report  
2     1.4        Match Report  
3    -1.0        Match Report  
4     2.7        Match Report  

[5 rows x 26 columns]

In [13]:
shooting.columns = shooting.columns.droplevel()

In [14]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [15]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-08-13,21:00,La Liga,Matchweek 1,Sat,Home,D,0,0,Rayo Vallecano,...,4-3-3,Alejandro Hernández,Match Report,NaN,21,5,17.0,1.0,0,0
1,2022-08-21,22:00,La Liga,Matchweek 2,Sun,Away,W,4,1,Real Sociedad,...,3-2-4-1,José Luis Munuera,Match Report,NaN,15,7,14.6,0.0,0,0
2,2022-08-28,19:30,La Liga,Matchweek 3,Sun,Home,W,4,0,Valladolid,...,4-3-3,Ricardo de Burgos,Match Report,NaN,24,9,14.4,1.0,0,0
3,2022-09-03,21:00,La Liga,Matchweek 4,Sat,Away,W,3,0,Sevilla,...,4-3-3,Antonio Matéu Lahoz,Match Report,NaN,18,5,16.0,2.0,0,0
4,2022-09-07,21:00,Champions Lg,Group stage,Wed,Home,W,5,1,cz Viktoria Plzeň,...,4-3-3,Lawrence Visser,Match Report,NaN,20,10,16.2,0.0,0,0


In [16]:
years = list(range(2023,2022, -1))
all_matches = []

In [17]:
url = ('https://fbref.com/en/comps/12/La-Liga-Stats')

In [18]:
import time
for year in years:
    data = requests.get(url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "La Liga"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [19]:
len(all_matches)

20

In [20]:
match_df = pd.concat(all_matches)

In [21]:
match_df.columns = [c.lower() for c in match_df.columns]

In [22]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2022-08-13,21:00,La Liga,Matchweek 1,Sat,Home,D,0,0,Rayo Vallecano,...,Match Report,NaN,21.0,5.0,17.0,1.0,0,0,2023,Barcelona
1,2022-08-21,22:00,La Liga,Matchweek 2,Sun,Away,W,4,1,Real Sociedad,...,Match Report,NaN,15.0,7.0,14.6,0.0,0,0,2023,Barcelona
2,2022-08-28,19:30,La Liga,Matchweek 3,Sun,Home,W,4,0,Valladolid,...,Match Report,NaN,24.0,9.0,14.4,1.0,0,0,2023,Barcelona
3,2022-09-03,21:00,La Liga,Matchweek 4,Sat,Away,W,3,0,Sevilla,...,Match Report,NaN,18.0,5.0,16.0,2.0,0,0,2023,Barcelona
5,2022-09-10,18:30,La Liga,Matchweek 5,Sat,Away,W,4,0,Cádiz,...,Match Report,NaN,16.0,8.0,14.9,0.0,0,0,2023,Barcelona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,2023-04-26,22:00,La Liga,Matchweek 31,Wed,Away,L,0.0,1.0,Celta Vigo,...,Match Report,NaN,12.0,1.0,16.7,1.0,0,0,2023,Elche
34,2023-04-29,16:15,La Liga,Matchweek 32,Sat,Home,W,4.0,0.0,Rayo Vallecano,...,Match Report,NaN,18.0,9.0,15.2,1.0,0,0,2023,Elche
35,2023-05-02,19:30,La Liga,Matchweek 33,Tue,Away,L,1.0,2.0,Almería,...,Match Report,NaN,14.0,1.0,14.0,0.0,0,0,2023,Elche
36,2023-05-14,16:15,La Liga,Matchweek 34,Sun,Home,W,1.0,0.0,Atlético Madrid,...,Match Report,NaN,15.0,4.0,13.4,0.0,0,0,2023,Elche


In [23]:
match_df.to_csv("matches.csv")